### Building Hybrid RAG + Redis + LLM

Hybrid - Lexical match + Semantic match 

Lexical - bm25

Semantic - Embedding (SentenceTransformer)


In [ ]:
#!pip install pymupdf
#!pip install faiss-cpu
#!pip install python-dotenv
#!pip install nltk
!#pip install rank-bm25

'#pip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
print("hi")

hi


In [ ]:
#!pip uninstall fitz -y
#!pip uninstall PyMuPDF -y
#!pip uninstall pymupdf -y

In [2]:
!pip install redis

  Using cached redis-6.4.0-py3-none-any.whl.metadata (10 kB)
Using cached redis-6.4.0-py3-none-any.whl (279 kB)


In [ ]:
import fitz  # PyMuPDF
import re
import langchain_text_splitters
import faiss
import redis
import numpy as np
import nltk
import os
import google.generativeai as genai

from dotenv import load_dotenv
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize,sent_tokenize

from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence-transformers import SentenceTransformer



ModuleNotFoundError: No module named 'faiss'

#### Verify pymupdf

In [4]:
print(fitz.__file__)
print(fitz.__version__ if hasattr(fitz, '__version__') else 'No version')
print(dir(fitz))

c:\Work Drive-Suchi\Mithileshsir_AI_ML_GenAI\Project\by me\ChatBot_v2\env-chatbot_v2\lib\site-packages\fitz\__init__.py
1.26.4
['ASSERT_PDF', 'Annot', 'AnyType', 'Archive', 'Base14_fontdict', 'Base14_fontnames', 'ByteString', 'CS_CMYK', 'CS_GRAY', 'CS_RGB', 'CheckColor', 'CheckFont', 'CheckFontInfo', 'CheckMarkerArg', 'CheckMorph', 'CheckParent', 'CheckQuad', 'CheckRect', 'ColorCode', 'Colorspace', 'ConversionHeader', 'ConversionTrailer', 'DeviceWrapper', 'DisplayList', 'Document', 'DocumentWriter', 'EMPTY_IRECT', 'EMPTY_QUAD', 'EMPTY_RECT', 'ENSURE_OPERATION', 'EPSILON', 'ElementPosition', 'EmptyFileError', 'FLT_EPSILON', 'FZ_MAX_INF_RECT', 'FZ_MIN_INF_RECT', 'FZ_RECOMPRESS_FAX', 'FZ_RECOMPRESS_J2K', 'FZ_RECOMPRESS_JPEG', 'FZ_RECOMPRESS_LOSSLESS', 'FZ_RECOMPRESS_NEVER', 'FZ_RECOMPRESS_SAME', 'FZ_SUBSAMPLE_AVERAGE', 'FZ_SUBSAMPLE_BICUBIC', 'FileDataError', 'FileNotFoundError', 'FitzDeprecation', 'Font', 'Graftmap', 'INFINITE_IRECT', 'INFINITE_QUAD', 'INFINITE_RECT', 'INVALID_NAME_CHARS

#### nltk download for tokenizer : BM25

In [5]:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\snuin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\snuin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

#### Redis server -client connection

In [ ]:
#redis_client = redis.Redis(host='localhost', port=6379, db=0)

In [ ]:
#print(redis_client.ping())  ## verfiy redis-server is running

True


In [ ]:
from dotenv import load_dotenv
import redis
import os

load_dotenv()

redis_client = redis.Redis(
                host = os.getenv("REDIS_HOST"),
                port = int(os.getenv("REDIS_PORT")),
                password = os.getenv("REDIS_PASSWORD"),
                ssl=True
                )

redis_client.set('foo', 'bar')
value = redis_client.get('foo')
print(value.decode()) ## if will not use decode then print byte str

bar


In [ ]:
print(redis_client.ping()) ## verify its working 

True


#### Gemini key Loaded

In [ ]:
load_dotenv()

# Your API Key setup
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("Google API Key is missing. Please set it in the .env file.")

# Configure the Generative AI library with the API key
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("api key configured")


api key configured


#### PDF extracted 

In [130]:
#doc = fitz.open("CleanBot_Robotic_Vacuum_Cleaner_FAQ.pdf")
doc = fitz.open("AMFI_MF.pdf")
print(doc)
text = ""
skip_keywords = [ "disclaimer", "table of contents", ]

for i, page in enumerate(doc):

    page_text = page.get_text("text")

    if i == 0:                          ## Specifically add this to avoid first page which is having disclaimer also
        # print("First page preview:\n")
        # print(doc[0].get_text("text"))
        continue

    if any(kw in page_text for kw in skip_keywords):
        continue        

    text += page_text

Document('AMFI_MF.pdf')


In [131]:
text

'Introduction \n \nDifferent investment avenues are available to investors. Mutual funds also offer good investment \nopportunities to the investors. Like all investments, they also carry certain risks. The investors \nshould compare the risks and expected returns after adjustment of tax on various instruments \nwhile taking investment decisions. The investors may seek advice from experts while making \ninvestment decisions. \nWith an objective to make the investors aware of functioning of mutual funds, an attempt has been \nmade to provide information in question-answer format which may help the investors in taking \ninvestment decisions. \n1) What is a Mutual Fund? \n \nMutual fund is a mechanism for pooling money by issuing units to the investors and investing \ninvestors funds in securities in accordance with objectives as disclosed in offer document. \nInvestments in securities are spread across a wide cross-section of industries and sectors and \nthus the risk is diversified beca

In [132]:
import re

# collapse all whitespace to single space
documents = re.sub(r'\s+', ' ', text)

# remove isolated characters (except numbers or words)
documents = re.sub(r'\b\w\b', '', documents)

documents = documents.strip()


In [133]:
documents

'Introduction Different investment avenues are available to investors. Mutual funds also offer good investment opportunities to the investors. Like all investments, they also carry certain risks. The investors should compare the risks and expected returns after adjustment of tax on various instruments while taking investment decisions. The investors may seek advice from experts while making investment decisions. With an objective to make the investors aware of functioning of mutual funds, an attempt has been made to provide information in question-answer format which may help the investors in taking investment decisions. ) What is  Mutual Fund? Mutual fund is  mechanism for pooling money by issuing units to the investors and investing investors funds in securities in accordance with objectives as disclosed in offer document. Investments in securities are spread across  wide cross-section of industries and sectors and thus the risk is diversified because all stocks may not move in the s

In [134]:
len(documents)

44143

In [135]:
import langchain_text_splitters

In [136]:
## character-based chunks with overlap:

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50,
                                          separators=["\n\n", "\n", " ", ""])
chunks = splitter.split_text(documents)


print(f"Total chunks: {len(chunks)}")
print(chunks[0])  # preview first chunk

Total chunks: 99
Introduction Different investment avenues are available to investors. Mutual funds also offer good investment opportunities to the investors. Like all investments, they also carry certain risks. The investors should compare the risks and expected returns after adjustment of tax on various instruments while taking investment decisions. The investors may seek advice from experts while making investment decisions. With an objective to make the investors aware of functioning of mutual funds, an


In [137]:
from sentence_transformers import SentenceTransformer

In [138]:
## Create embeddings

model = SentenceTransformer("all-MiniLM-L6-v2")
dense_embedding = model.encode(chunks, convert_to_numpy=True).astype('float32')  # FAISS requires float32

dense_embedding

array([[ 0.07045498, -0.01928587, -0.02310016, ..., -0.04449525,
         0.02631509,  0.05331701],
       [ 0.05381014, -0.00355125, -0.04846382, ..., -0.00643085,
         0.03483032,  0.01452152],
       [ 0.09488978, -0.06614771, -0.05396507, ..., -0.03850112,
         0.03113568, -0.0027707 ],
       ...,
       [ 0.0017096 ,  0.03545368, -0.02141625, ..., -0.07993176,
        -0.05482747,  0.03807379],
       [ 0.02352829,  0.05688978, -0.05820875, ..., -0.04374652,
        -0.05600811, -0.00170038],
       [ 0.00062163,  0.06241807, -0.02467038, ..., -0.0301974 ,
         0.00135569,  0.00899023]], dtype=float32)

In [139]:
dense_embedding.shape


(99, 384)

In [140]:
# Build FAISS index
import faiss

dim = dense_embedding.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(dense_embedding)

#### Setup for Sparse (Lexical matching) Retrival

In [141]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize,sent_tokenize

## Note: Sentence tokenized created 352 sentences from documents

tokenized_corpus = word_tokenize(documents) ## --> this is for documents 
print("total word after tokenized: ", len(tokenized_corpus))

## Preprocess chunks for BM25 (tokenize each chunk) --> because for Dense I used chunks so for sparse also I used chunks

tokenized_chunks = [word_tokenize(i) for i in chunks]

print("tokenized_chunks : ", len(tokenized_chunks))

bm25 = BM25Okapi(tokenized_chunks)

total word after tokenized:  7978
tokenized_chunks :  99


In [142]:
bm25

In [167]:
# Semantic retrieval (Dense)
def retrieve_dense(query, k=3):
    query_emb = model.encode([query])
    D, I = index.search(np.array(query_emb), k)
    print ("Dense distance: ", D)
    return I[0]  # return indices only

#### BM25 is not a binary or count-based metric — it's a ranking function that computes weighted relevance scores using floats. So it's expected that scores are floating-point numbers.

##### Ideally score array will have shape(99,). Each value is the BM25 score indicating how relevant that chunk is to the qyery. My chunk shape is 99


In [ ]:
# Lexical retrieval (Sparse using BM25)
def retrieve_sparse(query, k=3):
    tokenized_query = word_tokenize(query)
    scores = bm25.get_scores(tokenized_query)
    print(scores.shape) 
    I = np.argsort(scores)[::-1][:k]
    
    return I  # indices

In [156]:
## Hybrid retrieval logic (combine indices):

def hybrid_retrieve(query, k_dense=3, k_sparse=3):
    dense_indices = retrieve_dense(query, k_dense)
    print("dense_indicies : ", dense_indices)
    sparse_indices = retrieve_sparse(query, k_sparse)
    print("sparse_indicies : ", sparse_indices)

    # Union of indices (or you can do weighted fusion if desired)
    hybrid_indices = list(set(dense_indices) | set(sparse_indices))
    print("hybrid_indices : ", hybrid_indices)

    # Get the actual documents (from chunks/doc_texts/etc.)
    retrieved_docs = [chunks[i] for i in hybrid_indices]  # or doc_texts[i]
    print("retrieved_docs : ", retrieved_docs)
    return retrieved_docs

In [161]:
## Now use LLM

def generate_with_gemini(query, context_str):
    model = genai.GenerativeModel("gemini-2.5-flash")

   
    # Prepare the prompt
    prompt = f"""
                You are a financial assistant AI. Use only the context provided below to answer the user query clearly and concisely.

                Important rules:
                - ONLY use the context below to answer.
                - DO NOT include any disclaimers or headers.
                - If the answer is not present in the context, respond with: "I'm sorry, I don't have enough information to answer that."


    CONTEXT:
    {context_str}

    USER QUESTION:
    {query}

    Answer:
    """

    # Generate response
    response = model.generate_content(prompt)
    print("Response from LLM --> ", response.text.strip())
    return response.text.strip()


In [162]:
# Answer Generation Function (can be improved later):

def generate_answer(query, context):
    context_str = "\n---\n".join(context)
    
    # Call Gemini LLM with query + context
    answer = generate_with_gemini(query, context_str)
    
    #return f"Based on the following context:\n{context_str}\n\nAnswer: {query} → [You can now call your LLM here]"
    return answer



In [163]:
# Final RAG with cache (use hybrid retrieval):

def rag_with_cache(query, k_dense=3, k_sparse=3):
    # Step 1: Check Redis cache
    cached_answer = redis_client.get(query)
    if cached_answer:
        print("[Answer from Redis] :")
        return f"[CACHE HIT] {cached_answer.decode()}"
    
    # Step 2: Hybrid Retrieval (combine dense + sparse)
    context = hybrid_retrieve(query, k_dense, k_sparse)

    # Step 3: Generate answer
    answer = generate_answer(query, context)

     # Step 4: Cache the answer
    redis_client.set(query, answer, ex=60)  # cache for 60s
    return f"[RAG GENERATED] {answer}"

In [164]:
query= "what is cut off timing and its role in determining the applicable NAV for transactions in Mutual Fund schemes?"

answer = rag_with_cache(query)  ## --> hybrid Retrival and LLM response
print(answer)

dense_indicies :  [13 11 12]
(99,)
sparse_indicies :  [13 16 17]
hybrid_indices :  [np.int64(16), np.int64(17), np.int64(11), np.int64(12), np.int64(13)]
retrieved_docs :  ['the next business day; \uf0b7 Where the application is received after .00 pm – the closing NAV of the next business day. Equity oriented funds and debt funds (except liquid and overnight funds) . Purchases and Switch in . Redemptions and switch out .00 pm \uf0b7 If application is received and funds (for purchase) are available for utilization before cut off time, the closing NAV of the day on which the application is received. \uf0b7 If the application is received after cut off time, the closing NAV of the', 'after cut off time, the closing NAV of the next business day will be applicable. \uf0b7 If funds (for purchase) are not available for utilization, irrespective of the receipt time of application, closing NAV of the day on which the funds are available for utilization will be applicable. Examples: Assume that i

#### Run some query which totally mismatch like 
query = "how to cook pulav, give me receipe"

finally LLM respose as instructed : sorry , I dont have answer... but when I check Dense and Sparse it return indices like :

Dense distance:  [[1.9305966 1.9397118 1.9704973]]
dense_indicies :  [97 46 36]
sparse_indicies :  [55 83 36]

1- FAISS is a nearest neighbor search engine. It must return k results, even if the similarity is near zero.

2- So even if you're asking about "cooking pulav", it might return chunks that mention “process,” “steps,” or just have a similar sentence structure.

Using L2 (Euclidean) distance from FAISS, and the values are:
~1.93, ~1.94, ~1.97

### In dense retrieval, lower distance = more similar

These distances are very high, meaning:

➤ 🔴 The query is very far from your stored chunks

→ In other words: nothing matched semantically


In [168]:
##query ="what is open-ended schemes "
query = "how to cook pulav with soya chunks like fried-rice ?"
# print(query)
answer = rag_with_cache(query)
print(answer)


Dense distance:  [[1.9305966 1.9397118 1.9704973]]
dense_indicies :  [97 46 36]
(99,)
sparse_indicies :  [55 83 36]
hybrid_indices :  [np.int64(97), np.int64(83), np.int64(36), np.int64(55), np.int64(46)]
retrieved_docs :  ['investors can approach SEBI for redressing their complaints. Investor may lodge their complaints through SCORES (SEBI Complaints Redress System – https://scores.sebi.gov.in) or by sending their complaints on the address given below: Securities and Exchange Board of India Office of Investor Assistance and Education, Securities and Exchange Board of India, SEBI Bhavan II PN-C7,  Block BKC, Bandra Kurla Complex, Bandra East, Mumbai, Maharashtra 400051 Investors can lodge their complaint through', 'at the end of the year. Investors can compare the performance of their schemes with those of other mutual funds under the same category. They can also compare the performance of equity oriented schemes with the benchmarks like BSE Sensitive Index, Nifty, etc. 37) How to know

In [ ]:
queries = ["Suggest different types of funds available " ,
           "what is Balanced or hybrid funds",
           "who is Authorities to handle mf" ,
           "how to start investing in MF industries",
           "give me some introduction about mf",
           "what is Balanced or hybrid funds",
            "what is FoF Scheme" ]

print(" *** Demo *** RAG+CAG with or without Latency \n ")


for q in queries:
    print(f"\nQ: {q}")  
    answer = rag_with_cache(q)
    print(answer)

 *** Demo *** RAG+CAG with or without Latency 
 

Q: Suggest different types of funds available 
Query from function:  Suggest different types of funds available 
[CACHE HIT] Based on context: Pharmaceuticals etc. These funds are launched to get the benefit from growth and performance of the particular sector . For example, technology sector would primarily invest in tech companies only. The returns in these funds are dependent on the performance of t he respective sectors/industry. Thematic funds invest in stocks across a specific theme. These funds focus on specific themes or trends in the market. The themes can be based on economic trends, social changes, technological advancements etc. Examples of these funds are MNC funds, PSU (public sector units) funds, energy funds . Compare d to sectoral fund thematic funds 
Answer: Suggest different types of funds available  → Pharmaceuticals etc. These funds are launched to get the benefit from growth and performance of the particular sector

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 35.575945868s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
]